- n_x: the size of the input layer
- n_h: the size of the hidden layer (set this to 4) 
- n_y: the size of the output layer
J=−1m∑i=0m(y(i)log(a[2](i))+(1−y(i))log(1−a[2](i)))

In [2]:
import numpy as np

In [31]:
import pandas as pd
import numpy as np
from sklearn import svm, cross_validation , preprocessing
from sklearn.linear_model import LinearRegression
#file_name=r"...\train_data.csv"
data_frame=pd.read_csv(file_name,header=0,names=None,index_col=None,usecols=None,prefix =None,
                       dtype={0: np.int32, 1: np.int32, 2: np.float16,3: np.int32},skiprows=None,keep_default_na=True,
                       verbose=True
                      )
features=["total_rooms","population","median_income"]
target=["median_house_value_in_range"]
x=np.array(data_frame.drop(target,1))
y=np.array(data_frame[target])
x=preprocessing.scale(x)
x_train , x_test , y_train , y_test =cross_validation.train_test_split(x,y,test_size=.2)

C:\Users\Ravi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Tokenization took: 4.00 ms
Type conversion took: 8.99 ms
Parser memory cleanup took: 0.00 ms


In [33]:
print (x_train.shape)
print (y_train.shape)

(8599, 3)
(8599, 1)


In [3]:
def tanh(x):
    return np.tanh(x)

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [37]:
def initialize_weights(x,n_h,y):
    m=len(x[0])
    n=len(y[0])
    w1=np.random.randn(n_h,m)*0.01
    b1=np.zeros((n_h,1))
    w2=np.random.randn(n,n_h)*0.01
    b2=np.zeros((n,1))
    param={
        "w1":w1,
        "b1":b1,
        "w2":w2,
        "b2":b2,
    }
    return param

In [122]:
def propagate(param,x,y):
    w1=param["w1"]
    b1=param["b1"]
    w2=param["w2"]
    b2=param["b2"]
    z1=np.dot(w1,x.T)+b1
    a1=tanh(z1)
    z2=np.dot(w2,a1)+b2
    a2=sigmoid(z2)
    #print(y.shape)
    #print(a2.shape)
    m=len(x)
    cost=-(np.dot(y.T,np.log(a2).T)+np.dot(1-y.T,np.log(1-a2).T))/m
    cost = np.squeeze(cost)
    #print  (a2.shape)
    #print (cost)
    cache={
        "z1":z1 ,
        "a1":a1 ,
        "z2":z2 ,
        "a2":a2
    }
    return cost , cache , a2
    

In [123]:
param=initialize_weights(x_train,4,y_train)
cost , cache , a2=propagate(param,x_train,y_train)
#print (x_train.shape)
#print (y_train[0])
#print (param["w1"])
#print (param["b1"])
#print (param["w2"])
#print (param["b2"])
#print (param["w1"].shape)
#print (param["b1"].shape)
#print (param["w2"].shape)
#print (param["b2"].shape)


In [146]:
def backward_prop(param,cache,x,y):
    w1=param["w1"]
    b1=param["b1"]
    w2=param["w2"]
    b2=param["b2"]
    
    z1=cache["z1"]
    a1=cache["a1"]
    z2=cache["z2"]
    a2=cache["a2"]
    
    dz2=a2-y.T
    #print  (a1.shape)
    #print  (a2.shape)
    #print  (y.shape)
    #print  (dz2.shape)
    m=len(x)
    
    dw2=np.dot(dz2,a1.T)/m
    #print  (dw2.shape)
    db2=np.sum(dz2,axis=1,keepdims=True)/m
    #print  (db2.shape)
    #print  (w2.shape)
    #print  (dz2.shape)
    dz1=np.dot(w2.T,dz2)*(1-np.power(a1,2))
    dw1=np.dot(dz1,x)/m
    db1=np.sum(dz1,axis=1,keepdims=True)/m
    #print  (a2)
    gd={
        "dw1":dw1 ,
        "db1":db1 ,
        "dw2":dw2 ,
        "db2":db2 
    }
    return gd

In [147]:
backward_prop(param,cache,x_train,y_train)

{'dw1': array([[  -92.33248055,     8.66926505,  -388.9540936 ],
        [ -278.74239873,    23.98008338, -1161.84623733],
        [  131.0244326 ,   -11.17273616,   548.16527518],
        [  107.96622297,    -8.99204366,   450.29870309]]),
 'db1': array([[-1002.02138017],
        [-2996.56933186],
        [ 1409.31226701],
        [ 1157.8699882 ]]),
 'dw2': array([[-1256.291644  ,  1514.97848294,  -411.45936312,  -459.00395967]]),
 'db2': array([[-203088.08995292]])}

In [148]:
def update_parametres(param,gd,learning_rate=.01):
    w1=param["w1"]
    b1=param["b1"]
    w2=param["w2"]
    b2=param["b2"]
    
    dw1=gd["dw1"]
    db1=gd["db1"]
    dw2=gd["dw2"]
    db2=gd["db2"]
    
    w1=w1-learning_rate*dw1
    b1=b1-learning_rate*db1
    w2=w2-learning_rate*dw2
    b2=b2-learning_rate*db2
    
    param={
        "w1":w1,
        "b1":b1,
        "w2":w2,
        "b2":b2,
    }
    return param

In [149]:
def nn_model(x, y, n_h, num_iterations = 10):
    param=initialize_weights(x,n_h,y)
    for i in range(num_iterations):
        cost , cache , a2=propagate(param,x,y)
        gd=backward_prop(param,cache,x,y)
        param=update_parametres(param,gd,learning_rate=.01)
    
    return param

In [150]:
nn_model(x_train,y_train, 4, num_iterations=100)

C:\Users\Ravi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
C:\Users\Ravi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in add
  del sys.path[0]


{'w1': array([[  10466.39558732,   -5006.71838887,   83802.16604482],
        [ -24294.40424977,  -22089.47019934,   -4912.89011206],
        [ -64343.40458186,  -29724.58539718, -117114.92040909],
        [  45670.00758343,   26151.72424446,  109518.92727642]]),
 'b1': array([[  76839.97805955],
        [  -5331.73746425],
        [-108945.54084261],
        [  95100.8181959 ]]),
 'w2': array([[ 163990.98514425,    4757.60847937, -132396.97322314,
          141412.66232548]]),
 'b2': array([[203087.72931213]])}

In [168]:
def predict(param,x,y):
    cost , cache , prediction = propagate(param,x,y)
    prediction=(prediction>.45)
    return prediction

In [152]:
file_name=r"....\train.csv"
data_frame=pd.read_csv(file_name,header=0,names=None,index_col=None,usecols=None,prefix =None,
                    skiprows=None,keep_default_na=True,
                       verbose=True
                      )

Tokenization took: 0.00 ms
Type conversion took: 12.00 ms
Parser memory cleanup took: 0.00 ms


In [156]:
x=data_frame[['PassengerId','Pclass','Age']]
y=data_frame[['Survived']]
x=x.apply(lambda x: x.fillna(int(x.mean())),axis=0)
x = x.values
y = y.values

print (y)

[[0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]


In [160]:
param=nn_model(x,y, 4, num_iterations=100)

In [159]:
file_name=r"....\train.csv"
data_frame_test=pd.read_csv(file_name,header=0,names=None,index_col=None,usecols=None,prefix =None,
                    skiprows=None,keep_default_na=True,
                       verbose=True
                      )
x_test=data_frame[['PassengerId','Pclass','Age']]
y_test=data_frame[['Survived']]
x_test=x_test.apply(lambda x_test: x_test.fillna(int(x_test.mean())),axis=0)
x_test = x_test.values
y_test = y_test.values

Tokenization took: 0.00 ms
Type conversion took: 8.00 ms
Parser memory cleanup took: 0.00 ms


In [170]:
predictions = predict(param, x_test,y_test)
#print (predictions.T)
print ('Accuracy: %d' % float((np.dot(y_test.T,predictions.T) + np.dot(1-y_test.T,1-predictions.T))/float(y_test.size)*100) + '%')

Accuracy: 61%
